In [ ]:
!pip install pyforest

In [ ]:
import os
import pyforest

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
data  = pd.read_csv("/kaggle/input/playstore-dataset/playstore-analysis.csv")

data.head(3)

In [ ]:
data.shape


In [ ]:
data.describe()

In [ ]:
data.columns

In [ ]:
#Jacinta - The columns you are trying to drop are not available in your datafroame

# Drop specified columns from the DataFrame
data.drop(["App", "Current Ver", "Last Updated", "Current Ver"], axis=1, inplace=True)

# Display information about the DataFrame
data.info()

In [ ]:
data.columns

In [ ]:
data = data.drop_duplicates(keep = False, inplace = True)

In [ ]:
# data.shape